In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from RDtools2 import *
from scipy.linalg import inv,pinv,null_space
import pickle as pkl

In [ ]:
# here we will pass the diagonal equivalent reward matrix Q to the function
def getAvgLambdaStar(Q,px):
  return np.sum(np.log(np.diag(Q)) * px)

def hx(px):
  return np.sum([phi(i) for i in px])

In [ ]:
R = np.array([[3,0,0],[0,3,0],[0,0,3]])
s = np.array([[0.7,0.15,0.15],[0.15,0.7,0.15]])
px = np.array([0.333,0.334,0.333])

Dxy = getFullDistortionFunction(R,s)
result = getRD(px,Dxy,numPoints=400,show_pb=True)

# line for the bound
dmin = np.min(result['Dmax_v'])
dmax = np.max(result['Dmax_v'])
L = getAvgLambdaStar(R,px)
D_v = np.array([dmin,dmax])
B_v = hx(px) - D_v - L

# point of equality
#pxy = joint_p_from_pxgy_and_px(getT(R,s),px)
#Dstar = np.sum(pxy * d)
#Rstar = binent(px) - Dstar - L

In [ ]:
s2 = np.vstack((s,np.array([0.15,0.15,0.7])))
Dxy = getFullDistortionFunction(R,s2)
result2 = getRD(px,Dxy,numPoints=400,show_pb=True)

# line for the bound
dmin = np.min(result2['Dmax_v'])
dmax = np.max(result2['Dmax_v'])
L = getAvgLambdaStar(R,px)
D_v2 = np.array([dmin,dmax])
B_v2 = hx(px) - D_v2 - L

In [ ]:
s3 = np.vstack((s2,np.array([0.4,0.3,0.3])))
Dxy = getFullDistortionFunction(R,s3)
result3 = getRD(px,Dxy,numPoints=400,show_pb=True)

# line for the bound
dmin = np.min(result3['Dmax_v'])
dmax = np.max(result3['Dmax_v'])
L = getAvgLambdaStar(R,px)
D_v3 = np.array([dmin,dmax])
B_v3 = hx(px) - D_v3 - L

In [ ]:
S = s3

py0 = px @ pinv(S) # initial solution
v = null_space(S.T)[:,0] # null space vector

# find ranges for each coordinate
a = [-py0[i]/v[i] for i in range(len(py0))]
b = [(1-py0[i])/v[i] for i in range(len(py0))]
mins = [np.min([a[i],b[i]]) for i in range(len(py0))]
maxs = [np.max([a[i],b[i]]) for i in range(len(py0))]

# the safe region is the max of the minima to the min of the maxima
r = np.array([np.max(mins),np.min(maxs)])

# find distortions
Dxy = -np.log(S @ R)
dist = np.zeros(2)
rate = np.zeros(2)
for i in [0,1]:
    pyq = py0 + r[i]*v
    pxy = np.diag(pyq) @ S
    rate[i] = MI(pxy)
    dist[i] = np.sum(pxy * Dxy)